# Legalis Model - Prediction and Datascience around German Court cases

## Importing Libraries and Dataset from Huggingface

In [1]:
from datasets import load_dataset
import re
import locale

dataset=load_dataset("LennardZuendorf/legalis")

dataset['train'].features

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/41791 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4644 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to C:/Users/lenna/.cache/huggingface/datasets/LennardZuendorf___parquet/LennardZuendorf--legalis-0a81af9488284743/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'id': Value(dtype='int64', id=None),
 'court': {'id': Value(dtype='int64', id=None),
  'jurisdiction': Value(dtype='string', id=None),
  'level_of_appeal': Value(dtype='string', id=None),
  'name': Value(dtype='string', id=None),
  'state': Value(dtype='int64', id=None)},
 'file_number': Value(dtype='string', id=None),
 'date': Value(dtype='timestamp[s]', id=None),
 'type': Value(dtype='string', id=None),
 'content': Value(dtype='string', id=None),
 'tenor': Value(dtype='string', id=None),
 'reasoning': Value(dtype='string', id=None)}

## filtered data for cases with a disputed amount, extracting value from dataset

In [2]:
def streitwert_filter(case):
    return 'streitwert' in case['content'].lower()

streitwert_dataset = dataset.filter(streitwert_filter)

print(streitwert_dataset)

Filter:   0%|          | 0/41791 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4644 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'court', 'file_number', 'date', 'type', 'content', 'tenor', 'reasoning'],
        num_rows: 14586
    })
    test: Dataset({
        features: ['id', 'court', 'file_number', 'date', 'type', 'content', 'tenor', 'reasoning'],
        num_rows: 1594
    })
})


In [9]:
# Extract `Streitwert` from case content
clean_html_pattern = re.compile('<.*?>')
locale.setlocale(locale.LC_ALL , 'de_DE.UTF-8')

#c = cases[4]
def extract_streitwert(case):

    case_content = case['content']
    case_content = re.sub(clean_html_pattern, '', case_content)
    pattern = re.compile('der streitwert wird auf ([\s0-9-\.,]*?) (eur|euro|€) festgesetzt')

    value = None
    for m in pattern.finditer(case_content.lower()):
        value = m.group(1).rstrip('-')

        try :
            value = locale.atof(value)
        except ValueError:
            value = None

    case['disputed_amount']=value
    return case
            
streitwert_dataset = streitwert_dataset.map(extract_streitwert)
streitwert_dataset = streitwert_dataset.filter(lambda data: data['disputed_amount'] is not None)

print(streitwert_dataset)

Map:   0%|          | 0/2307 [00:00<?, ? examples/s]

Map:   0%|          | 0/241 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2307 [00:00<?, ? examples/s]

Filter:   0%|          | 0/241 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'court', 'file_number', 'date', 'type', 'content', 'tenor', 'reasoning', 'disputed_amount'],
        num_rows: 2307
    })
    test: Dataset({
        features: ['id', 'court', 'file_number', 'date', 'type', 'content', 'tenor', 'reasoning', 'disputed_amount'],
        num_rows: 241
    })
})
